In [33]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import re
import tqdm
import librosa
from datasets import Dataset, load_metric
import json
import IPython.display as ipd
import numpy as np
import random
import torch
from src.data_util import DataCollatorCTCWithPadding
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer

root_dir = '/data/users/akrishnan/multilingual_asr/corpora/l2arctic'
speaker_info = pd.read_csv(f'{root_dir}/speaker_info.csv',sep='|')
speaker_info.columns = ['speaker','gender','accent','wav_files','annotations']
#SUBSAMPLING MODEL
# speaker_info = speaker_info[speaker_info["speaker"].isin(['YBAA','ZHAA','LXC','BWC','HQTV','THV','HJK','YKWK'])]
speaker_info = speaker_info[speaker_info["speaker"].isin(['YBAA','BDL'])]


speaker_files = {}
arctic_data = pd.DataFrame(['path','sentence','name','gender','accent','speaker'])
data_list=[]
for speaker in tqdm.tqdm(speaker_info.itertuples(index=False),total=len(speaker_info)):
    wav_transcript_dict = {}
    for wav in tqdm.tqdm_notebook(os.listdir(f'{root_dir}/{speaker.speaker}/wav/')):
        #Getting full paths for the wav transcript pairs
        wav_path = f"{root_dir}/{speaker.speaker}/wav/{wav}"
        transcript_path = f"{root_dir}/{speaker.speaker}/transcript/{wav.replace('.wav','.txt')}"
        
        #Reading transcripts and pairs. Note the forced resampling in librosa.load()
        array,sampling_rate=librosa.load(wav_path,sr=16000)
        transcript=open(transcript_path).read()
        
        data_dict={
            'audio': {'array' : array, 'sampling_rate':sampling_rate},
            'sentence' : transcript,
            'path' : wav,
            'gender' : speaker.gender,
            'accent' : speaker.accent,
            'speaker': speaker.speaker}
        
        data_list.append(data_dict)

df=pd.DataFrame(data_list)
#Making Huggingface dataset
l2_data=Dataset.from_pandas(df)

  0%|                                                                                                                                                                                 | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_12488/3566876723.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wav in tqdm.tqdm_notebook(os.listdir(f'{root_dir}/{speaker.speaker}/wav/')):


  0%|          | 0/1130 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                 | 0/2 [00:53<?, ?it/s]


KeyboardInterrupt: 

In [41]:
model_path="facebook/wav2vec2-base-100h"
model_path.split('/')[-1]

'wav2vec2-base-100h'

In [35]:
blha=6

In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import re
import tqdm
import librosa
from datasets import Dataset, load_metric
import json
import IPython.display as ipd
import numpy as np
import random
import torch
from src.data_util import DataCollatorCTCWithPadding
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer


comet_ml is installed but `COMET_API_KEY` is not set.


In [8]:
# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-base-100h",
#     attention_dropout=0.0,
#     hidden_dropout=0.0,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.0,
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),

model.name_or_path

'facebook/wav2vec2-base-100h'

In [17]:
wer_metric = load_metric("wer")
wer_metric.compute(predictions=["Hi"], references=["hi"])

1.0

In [31]:
df=pd.read_csv('corpora/l2arctic/l2_arctic_processed-Copy1.csv')
l2_data=Dataset.from_pandas(df)
l2_data['input_values'][1]

'[-0.10538473 -0.16507171 -0.13236764 ...  0.12238505  0.1155107\n  0.12516677]'

In [36]:

''' TRAINING WITH US ACCENTS '''

us_indices = np.where(np.array(l2_data['accent'])=='US')[0]
test_indices_accent= np.where(np.array(l2_data['accent'])!='US')[0]

#FOR TRAINING WE SPLIT THE DATA BY UTTERANCES FIRST (EACH SPEAKER DUPLICATES THE DATA)

#SELECTING UTTERANCES FOR SPLITS
US_Data=l2_data.select(us_indices)
utterance_ids=np.unique(US_Data['path'])
np.random.shuffle(utterance_ids)
train_utterances,val_utterances,test_utterances= np.split(utterance_ids,[int(0.7*len(utterance_ids)),int(0.8*len(utterance_ids))])

#COMPILING DATA USING UTTERANCES and ACCENT
train_indices=np.where((np.isin(l2_data['path'],train_utterances)) & (np.array(l2_data['accent'])=='US'))[0]
val_indices=np.where((np.isin(l2_data['path'],val_utterances)) & (np.array(l2_data['accent'])=='US'))[0]
test_indices_us=np.where((np.isin(l2_data['path'],test_utterances)) & (np.array(l2_data['accent'])=='US'))[0]

np.random.shuffle(train_indices)
np.random.shuffle(val_indices)
np.random.shuffle(test_indices_us)

# np.random.shuffle(indices)
# train_indices,val_indices,test_indices_us= np.split(indices,[int(0.8*len(indices)),int(0.9*len(indices))])

training_dataset = l2_data.select(train_indices)

validation_dataset = l2_data.select(val_indices)

test_indices=np.concatenate([test_indices_accent,test_indices_us])
testing_dataset = l2_data.select(test_indices)

In [3]:
!du -hs corpora/l2arctic

17G	corpora/l2arctic


In [1]:
from datasets import load_from_disk
l2_data=load_from_disk('corpora/l2arctic/l2_arctic_processed.hf')

ArrowInvalid: Tried reading schema message, was null or length 0

In [35]:
np.where((np.isin(l2_data['path'],train_utterances)) & (np.array(l2_data['accent'])=='US'))[0]

array([26868, 26872, 26874, ..., 31391, 31392, 31393])

In [2]:
indices = np.where(np.array(l2_data['accent'])=='US')[0]
test_indices_accent= np.where(np.array(l2_data['accent'])!='US')[0]

#FOR TRAINING WE SPLIT THE DATA BY UTTERANCES FIRST (EACH SPEAKER DUPLICATES THE DATA)

#SELECTING UTTERANCES FOR SPLITS
US_Data=l2_data.select(indices)
utterance_ids=np.unique(US_Data['path'])
np.random.shuffle(utterance_ids)
train_utterances,val_utterances,test_utterances= np.split(utterance_ids,[int(0.7*len(utterance_ids)),int(0.8*len(utterance_ids))])

#COMPILING DATA FOR UTTERANCES
train_indices=np.where(np.isin(US_Data['path'],train_utterances))[0]
val_indices=np.where(np.isin(US_Data['path'],val_utterances))[0]
test_indices_us=np.where(np.isin(US_Data['path'],test_utterances))[0]
np.random.shuffle(train_indices)
np.random.shuffle(val_indices)
np.random.shuffle(test_indices_us)


training_dataset = l2_data.select(train_indices)

validation_dataset = l2_data.select(val_indices)

test_indices=np.concatenate([test_indices_accent,test_indices_us])
testing_dataset = l2_data.select(test_indices)

In [2]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
test_accents=np.unique(testing_dataset['accent'])
for accent in test_accents:
    accent_indices=np.where(np.array(testing_dataset['accent'])==accent)[0]
    accent_test_dataset=testing_dataset.select(accent_indices)
    print(np.unique(accent_test_dataset['accent']))
#     accent_test_metrics=trainer.evaluate(accent_test_dataset,metric_key_prefix=accent)
#     print(accent_test_metrics)
#     wandb.log({f'{accent}_wer':accent_test_metrics[f'{accent}_wer']})
    


['Arabic']
['US']


In [11]:
processor.pad([{'input_features':np.array([1,1,1])},{'input_features':[1,1,1]}],return_tensors="pt")

ValueError: You should supply an instance of `transformers.BatchFeature` or list of `transformers.BatchFeature` to this method that includes input_values, but you provided ['input_features']

ERROR! Session/line number was not unique in database. History logging moved to new session 1547


In [2]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


''' TRAINING WITH US ACCENTS '''
indices = np.where(np.array(l2_data['accent'])=='US')[0]
test_indices= np.where(np.array(l2_data['accent'])!='US')[0]

#Splitting Val from train
np.random.shuffle(indices)
train_indices,val_indices= np.split(indices,[int(0.8*len(indices))])


data_collator = DataCollatorCTCWithPadding(processor = processor, padding=True)

training_dataset = l2_data.select(train_indices)

validation_dataset = l2_data.select(val_indices)

testing_dataset = l2_data.select(test_indices)



wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer*100}



model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
).to('cpu')

model.freeze_feature_extractor()



training_args = TrainingArguments(
    output_dir='l2_arctic_train/',
    group_by_length=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=100,
    gradient_checkpointing=True,
#    fp16=True,
    save_steps=400,
    eval_steps=400,
    logging_steps=400,
    learning_rate=3e-4,
    warmup_steps=500,
    save_total_limit=2,
    push_to_hub=False,
    metric_for_best_model = 'eval_wer',
    report_to='wandb',
    seed=123,
    load_best_model_at_end = True
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=processor.feature_extractor,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/users/akrishnan/miniconda3/envs/coref/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [34]:

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch
#validation_dataset = validation_dataset.map(prepare_dataset, remove_columns=['audio','sentence'])
testing_dataset= testing_dataset.map(prepare_dataset, remove_columns=['audio','sentence'])

0ex [00:00, ?ex/s]

/data/users/akrishnan/miniconda3/envs/coref/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [15]:
validation_loader=torch.utils.data.DataLoader(validation_dataset,collate_fn=data_collator)

In [22]:
x=trainer.evaluate(validation_dataset.select(range(10)),metric_key_prefix='Vietnamese')

/data/users/akrishnan/miniconda3/envs/coref/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [35]:
test_accents=np.unique(testing_dataset['accent'])
for accent in test_accents:
    accent_indices=np.where(np.array(testing_dataset['accent'])==accent)[0]
    accent_test_dataset=testing_dataset.select(accent_indices[:10])
    accent_test_metrics=trainer.evaluate(accent_test_dataset,metric_key_prefix=accent)
    print(accent_test_metrics)
    

{'Arabic_loss': 10.014276504516602, 'Arabic_wer': 100.0, 'Arabic_runtime': 5.9374, 'Arabic_samples_per_second': 1.684, 'Arabic_steps_per_second': 0.337}
{'Chinese_loss': 9.919207572937012, 'Chinese_wer': 100.0, 'Chinese_runtime': 6.6148, 'Chinese_samples_per_second': 1.512, 'Chinese_steps_per_second': 0.302}


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
''' TRAINING WITH US ACCENTS '''
indices = np.where(np.array(l2_data['accent'])=='US')[0]
test_indices= np.where(np.array(l2_data['accent'])!='US')[0]

#Splitting Val from train
np.random.shuffle(indices)
train_indices,val_indices= np.split(indices,[int(0.8*len(indices))])


data_collator = DataCollatorCTCWithPadding(processor = processor, padding=True)

training_dataset = l2_data.select(train_indices)

validation_dataset = l2_data.select(val_indices)

testing_dataset = l2_data.select(test_indices)


In [ ]:
np.where(np.array(l2_data['accent'])=='US')[0]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer*100}



# Text Preprocessing

### Cleaning

In [ ]:

chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch
l2_data = l2_data.map(remove_special_characters)

### Vocab Extraction

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}
vocab = l2_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=l2_data.column_names)
vocab_dict = {v: k for k, v in enumerate(sorted(vocab['vocab'][0]))}

#Accounting for blanks and unknowns
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

#Saving into file
with open(f'{root_dir}/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Setting up audio processors and tokenizers

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Pusing the audio and transcript through wav2vec tokenizers and processors

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch
l2_data = l2_data.map(prepare_dataset, remove_columns=['audio','sentence'])

# Defining Train Val Test Split

In [ ]:
indices = np.arange(len(l2_data))
np.random.shuffle(indices)
train_indices,val_indices,test_indices = np.split(indices,[int(0.7*len(indices)),int(0.8*len(indices))])

BATCH_SIZE=8

data_collator = DataCollatorCTCWithPadding(processor = processor, padding=True)

training_dataset = l2_data.select(train_indices)

validation_dataset = l2_data.select(val_indices)

testing_dataset = l2_data.select(test_indices)


# Setting up evaluation Metrics

In [ ]:
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Loading Model, setting up Training Arguments

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)
model.freeze_feature_extractor()

In [ ]:
training_args = TrainingArguments(
  output_dir='l2_arctic_train/',
  group_by_length=True,
  per_device_train_batch_size=BATCH_SIZE,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

## US-EN Subset

In [ ]:
# import shutil
# import os
# import tqdm
# root_dir = '/data/users/akrishnan/multilingual_asr/corpora/l2arctic/cmu_us_en/'
# for speaker in tqdm.tqdm(['BDL','CLB','RMS','SLT']):
#     old_file_dir=f'{root_dir}/cmu_us_{speaker.lower()}_arctic/wav/'
#     transcripts={i[1:-1].strip().split()[0]:" ".join(i[1:-1].strip().split()[1:])[1:-1] for i in open(f'{root_dir}/cmu_us_{speaker.lower()}_arctic/etc/txt.done.data').read().splitlines()}
#     for old_file in tqdm.tqdm_notebook(os.listdir(old_file_dir)):
#         old_wav_path=f'{old_file_dir}/{old_file}'
#         new_wav_path=f'{root_dir}/{speaker}/wav/{old_file}'
#         new_transcript_path=f"{root_dir}/{speaker}/transcripts/{old_file.replace('.wav','.txt')}"
#         try: 
#             x=transcripts[old_file.replace('.wav','')]
#         except:
#             print(f'Transcription for {old_file} not found')
#             continue
            
#         shutil.copyfile(src=old_wav_path, dst=new_wav_path)
#         with open(new_transcript_path,'w') as f:
#             f.write(transcripts[old_file.replace('.wav','')])


    




In [4]:
import numpy as np
x=np.ones(10)
y=np.zeros(20)
np.concatenate([x,y])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
np.savez('blah.npz',z=x,y=y)
np.load('blah.npz')['z']

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])